In [ ]:
# https://www.kaggle.com/datasets/atulanandjha/lfwpeople




The code below downloads all the necessary data from the COCO dataset.
It only downloads the "person" data, and it downloads it to the fiftyone location on the computer.

In [ ]:
import fiftyone as fiftyOne
import fiftyone.zoo as fiftyOneZoo

#Guard just to stop accidental download
download = False

if download:
    # Download the COCO 2017 train data
    dataset = fiftyOneZoo.load_zoo_dataset(
        "coco-2017",
        split="train",
        max_samples=None,          
        label_types=["detections"],
        classes=["person"]
    )
    
    # Download the COCO 2017 val data
    dataset = fiftyOneZoo.load_zoo_dataset(
        "coco-2017",
        split="validation",
        max_samples=None,          
        label_types=["detections"],
        classes=["person"]
    )
    # Download the COCO 2017 test data
    train_dataset = fiftyOneZoo.load_zoo_dataset(
        "coco-2017",
        split="test",
        max_samples=None,          #
        label_types=["detections"],
        classes=["person"]
    )


/Users/dariushtomlinson/Documents/Documents - Dariush’s MacBook Pro/Exeter/Computer Science/Year 3/Dissertation/Project/Pi-Alarm/Models/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defines the Classes needed for the data conversion

In [14]:
#Category dict where the key is the id, and the value is the category
#Allows for conversion from id to category
category_dict = {}

#Dict which contains all the Images
images_dict = {}

class Category:
    """
        Class which handles the details of each annotation class.
        Contains id, name and supercategory.

    """

    def __init__(self, id):
        """ Constructor for Category.

            Parameters:
                id (int): The id of the category
 
        """
        self.id = id
    
    def define_details(self, name, supercategory):
        """
        Setter which sets the name and supercategory.

        Parameters:
        name (string): The name of this category.
        supercategory string): The supercategory name of this Category.

        """
        self.name = name
        self.supercategory = supercategory

class Annotation:
    """
        Class which represents the Annotation (or label for one specific class) of an image.
        Contains bbox details, and sizing, and how to convert that from json to a .txt.
    """
    

    def __init__(self, id):
        """
            Constructor for the annotation.

            Parameters:
                id (int): The id of the annotation.
        """
        self.id = id

    def define_details(self,category_id,area, bbox, image_id):
        """
            Setter for the details of the annotation.

            Parameters:
                category_id (int): The id of the annotation.
                area (float): the area of the bbox
                bbox ([float]): coordinates of the bounding box for this annotation
                image_id (int): id of the image this annotation is for
            Raises:
                KeyError: If category_id is not a valid id.
        """
        #Guard statement to check category is an active one
        if category_id not in category_dict:
            raise KeyError(f"Category '{category_id}' not found in the dictionary")
        
        self.category = category_dict[category_id]
        self.area = area
        self.bbox = bbox
        self.image_id = image_id

    
    def get_yolo_line(self, image_width, image_height):
        """
            Uses the image dimensions and bbox dimensions to generate the normalised bbox values as yolo expects,
            and creates a text line as yolo expects for one of its labels.

            Parameters:
                image_width (int): The width of the image.
                image_height (int): The height of the image.

            Returns:
                str: A single line string formatted for YOLO annotation files:
                    "<class_id> <center_x> <center_y> <width> <height>"
                    where all coordinates and sizes are normalized floats.

        """
        #Coco bbox dimensions
        bbox_low_x = self.bbox[0]
        bbox_high_y = self.bbox[1]
        bbox_width = self.bbox[2]
        bbox_height = self.bbox[3]

        #Normalised bbox dimensions
        bbox_norm_x = (bbox_low_x + bbox_width / 2) / image_width
        bbox_norm_y = (bbox_high_y + bbox_height / 2) / image_height
        bbox_norm_width = bbox_width / image_width
        bbox_norm_height = bbox_height / image_height


        return f"{self.category.id} {bbox_norm_x} {bbox_norm_y} {bbox_norm_width} {bbox_norm_height}"


class Image:
    """ 
        A class representing one of the dataset images.
        Contains a list of associated attributes, and a link to the image file.
    
    """


    def __init__(self, id):
        """ 
            Constructor to define an Image with a given id.
        
            Parameters:
                id (int): The id of the Image.
        """
        self.image_id = id
        self.attribute_list = []
    
    def define_details(self, filename, width, height):
        """
            Setter for the details of the Images.

            Parameters:
                filename (string): The filename of the image.
                width (int): the width of the image
                height (int): the height of the image
        
        """
        self.filename = filename
        self.width = width
        self.height = height

    def add_attribute(self,attribute):
        """ 
            Adds an attribue to the list for this image.

            Parameters:
                attribute (Attribute): Attribute to be added.
        """
        self.attribute_list.append(attribute)
    
    def convert_to_yolo_line(self):
        """
            Gets all the details required for the label.
            
            Returns:
                list:
                    A two-element list containing:
                        [0] str: The filename of the image the annotations belong to.
                        [1] list of str: A list of YOLO annotation lines, one per object.
                            Each line has the format:
                            "<class_id> <center_x> <center_y> <width> <height>"


        """
        #Get all the annotations as lines to be appended in the file
        lines = []
        for annotation in self.attribute_list:
            lines.append(annotation.get_yolo_line(self.width, self.height))

        return [self.filename,lines]







In [ ]:
import os
import json

json_path = "Data/train&val/train/labels.json"
img_path = "Data/train&val/train/data/"

#Load the json from the file path into data
with open(json_path, 'r') as file:
    data = json.load(file)



#Loop through each category, creating a Category object, and append it to the category dict
for category_json in data["categories"]:
    #Create Category
    category = Category(category_json["id"])
    category.define_details(name=category_json["name"],supercategory=category_json["supercategory"])


    #Since I am only labelling people, I am doing a check to see if its the person class
    #Setting people category id to 0 as we are just dealing with it and yolo classes are 0 indexed.
    if category.name == "person":
        people_category_id = category.id
        category.id = 0

    #Append it
    category_dict[category.id] = category

    


#Since I am only labelling people, this is a guard to check that the people id has been noted
try:
    people_category_id
except NameError:
    raise ValueError("People category id is not set")

#Loop through every image, creating an Image object, and append it to the image dict
for images_json in data["images"]:
    #Create image
    image = Image(images_json["id"])
    image.define_details(filename=img_path+images_json["file_name"],width=images_json["width"],height=images_json["height"])

    #Append it
    images_dict[image.image_id] = image

#Loops through every annotation, creating an Annotation object, and add it to the appropriate Image object.
for annotation_json in data["annotations"]:
    #Checking if annotation 
    if annotation_json["category_id"] == people_category_id:
        #Create annotation
        annotation = Annotation(annotation_json["id"])

        #Putting 0 for the category as yolo labelling is 0 indexed and we are just dealing with people
        annotation.define_details(category_id=0, area=annotation_json["area"], bbox=annotation_json["bbox"], image_id=annotation_json["image_id"])
        
        #Add the annotation to the image
        images_dict[annotation.image_id].add_attribute(annotation)



In [ ]:
import os
import shutil


def empty_folder(folder_path):
    """
        Empties a folder and deletes all its contents including files and subfolders.

        Parameters:
            folder_path (string) : path of the folder to empty

    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isdir(file_path):
            shutil.rmtree(file_path) 
        else:
            os.remove(file_path) 

set_type = "train"

# Set the paths for both the images and labels of the dataset
images_path = "Data/yolo_dataset/images/" + set_type + "/"
labels_path = "Data/yolo_dataset/labels/" + set_type + "/"

#Create folders if they don't exist
os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

#Empty both folders
empty_folder(images_path)
empty_folder(labels_path)



count = 1
for image in images_dict.values():
    [image_name, label_lines] = image.convert_to_yolo_line()
    file_name = f"image{count}"
    with open(labels_path + file_name + ".txt", "w") as f:
        for line in label_lines:
            f.write(line + "\n")
    shutil.copy2(image.filename, images_path + file_name + ".png")



    count = count + 1



This code creates the yolo nano model if it doesn't exist currently. 

In [17]:
from ultralytics import YOLO

#Get nano model
model = YOLO('yolov8n.pt')

model.train(
    data="dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device= "mps"
)

Ultralytics 8.3.235 🚀 Python-3.11.5 torch-2.9.1 MPS (Apple M2 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fa

KeyboardInterrupt: 